<a href="https://colab.research.google.com/github/ParthikB/Neural-Style-Transfer/blob/v0.3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Uncomment the following code to mount the google drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%tensorflow_version 1.15.0 # Defining which Tf version to use
import tensorflow as tf
import keras.backend as K
from keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.python.keras import models
from keras.models import Model
from keras.preprocessing import image

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2
import os
import urllib
import statistics

In [32]:
# Enabling Eager Execution
tf.enable_eager_execution()
print("Eager Execution Initialized:",tf.executing_eagerly())

Eager Execution Initialized: True


# Helper Functions

### Selecting the Feature Layers needed

In [0]:
# Defining the Feature Layers we need respectively
styleLayers = ['block1_conv2', 
               'block2_conv2', 
               'block3_conv3', 
               'block4_conv3', 
               'block5_conv3']

contentLayer = ['block3_conv2']


numContentLayers = len(contentLayer) # Number of Content Layers
numStyleLayers   = len(styleLayers)  # Number of Style Layers

In [0]:
# Defining Function to import model (VGG19)
def getModel():

  # Loading Model from tf
  model = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
  model.trainable = False # Freezing to parameters

  # Features of the Respective Layers
  contentFeatures = [model.get_layer(name).output for name in contentLayer]
  styleFeatures   = [model.get_layer(name).output for name in styleLayers]
  
  modelOutput = contentFeatures + styleFeatures

  return models.Model(model.input, modelOutput)

In [0]:
# Defining GRAM MATRIX
def gram(x):

  # number of channels
  channels = int(x.shape[-1])
  
  # reshaping to channel first
  a = tf.reshape(x, [-1, channels])
  n = tf.shape(a)[0]
  
  # gram matrix
  gram = tf.matmul(a, a, transpose_a=True)
  
  return gram / tf.cast(n, tf.float32)


# Defining CONTENT COST
def contentCost(contentFeatures, generateFeatures):
  return tf.reduce_mean(tf.square(contentFeatures-generateFeatures))


# Defining STYLE COST
def styleCost(styleFeatures, generateFeatures):
  styleGram = gram(styleFeatures)
  return tf.reduce_mean(tf.square(styleGram - generateFeatures))

### Image Manipulations

In [0]:
"""
  Helper function to compute our content and style feature representations.
 
  This function will simply load and preprocess both the content and style 
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers. 
  
  Arguments:
    model: The model that we are using.
    content_path: The path to the content image.
    style_path: The path to the style image
    
  Returns:
    returns the style features and the content features. 
  """

def getFeatures(contentPath, stylePath, model):
  
  # Loading and pre-processing the images
  _, content = inputImageAndPreprocess(contentPath)
  _, style     = inputImageAndPreprocess(stylePath)

  # Defining the respective outputs from our model
  contentOutputs = model(content)
  styleOutputs = model(style)
  
  # Extracting out the different features from the model output
  contentFeatures = [contentFeature[0] for contentFeature in contentOutputs[numStyleLayers:]]
  styleFeatures = [styleFeature[0] for styleFeature in styleOutputs[:numStyleLayers]]

  return contentFeatures, styleFeatures


def loadImage(path_to_img):
  max_dim = 512
  img2show = Image.open(path_to_img)
  long = max(img2show.size)
  scale = max_dim/long
  img = img2show.resize((round(img2show.size[0]*scale), round(img2show.size[1]*scale)), Image.ANTIALIAS)
  
  img = image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img2show, axis=0)
  return img2show, img
 

def urlToImage(url):
  resp = urllib.request.urlopen(url)
  img = np.asarray(bytearray(resp.read()), dtype='uint8')
  img = cv2.imdecode(img, cv2.IMREAD_COLOR)
  img2show = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  img = np.expand_dims(img2show, axis=0)
  return img2show, img
 

def inputImageAndPreprocess(path):
  # Loading the image and reshaping it according to VGG19 requirements.
  if path[:5]=='https':
    print("Loading Image from Internet...")
    img2show, img = urlToImage(path)
  else:
    print("Loading Image from Local...")
    img2show, img = loadImage(path)
  
  # Preprocessing the img according to VGG19 requirements
  img = tf.keras.applications.vgg19.preprocess_input(img)

  return img2show, img


# Deprocessing Image to save locally
def deprocessImage(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")
  
  # perform the inverse of the preprocessiing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

### Computing the total Loss

In [0]:
def totalLoss(model, lossWeights, generateImage, contentFeatures, styleFeatures):

  # Extracting the respective weights
  contentWeight, styleWeight = lossWeights

  # Extracting the generate image features from the model
  modelOutputs = model(generateImage)

  # Splitting the generate Features into different categories
  contentGenerateFeatures = modelOutputs[numStyleLayers:]
  styleGenerateFeatures   = modelOutputs[:numStyleLayers]

  # Initializing all costs with 0
  contentCostValue, styleCostValue = 0, 0

  # Defining partial weights
  contentWeightPerLayer = 1.0 / float(numContentLayers)
  styleWeightPerLayer = 1.0 / float(numStyleLayers)

  # Computing Content Cost
  for generateContent, combinationContent in zip(contentFeatures, contentGenerateFeatures):
    contentCostValue += contentWeightPerLayer * contentCost(combinationContent[0], generateContent)

  # Computing Style Cost for every layer
  for generateStyle, combinationStyle in zip(styleFeatures, styleGenerateFeatures):
    styleCostValue += styleWeightPerLayer * styleCost(combinationStyle[0], generateStyle)
  

  # Assigning the weights
  contentCostValue *= contentWeight
  styleCostValue *= styleWeight

  # Computing the Total Loss
  totalLossValue = contentCostValue + styleCostValue

  return totalLossValue, contentCostValue, styleCostValue

### Computing the Gradient Descent

In [0]:
def computeGrads(config):
  with tf.GradientTape() as tape:
    allLoss = totalLoss(**config)

  loss = allLoss[0]

  return tape.gradient(loss, config['generateImage']), allLoss

In [0]:
"""
FUNTIONS DEFINED SO FAR:

  > getModel()
  > gram()
  > contentCost()
  > styleCost()
  > getFeatures()
  > load_img()
  > inputImageAndPreprocess()
  > totalLoss()
  > computeGrads()

"""

### Time to get it all together

In [0]:
# Defining the MAIN TRAINING FUNCTION
def runStyleTransfer(contentPath,
                     stylePath,
                     iterations     = 1000,
                     SAVE_EVERY     = 1,
                     contentWeight  = 1e3,
                     styleWeight    = 1e-2,
                     output_dirName = 'output'):
    
  # Importing the Model
  model = getModel()
  for layer in model.layers:
    layer.trainable = False

  # Extracting out the respective features from the model
  contentFeatures, styleFeatures = getFeatures(contentPath, stylePath, model)
  styleFeatures = [gram(styleFeature) for styleFeature in styleFeatures]

  # Creating the Generate Image
  _ , generateImage = inputImageAndPreprocess(contentPath)
  generateImage = tf.Variable(generateImage, dtype=tf.float32)

  # Defining the Adam Optimizer
  optimizer = tf.train.AdamOptimizer(learning_rate=5, epsilon=1e-3)

  # Storing the best Image and Loss
  bestLoss, bestImage = float('inf'), None

  # Zipping the Weights
  lossWeights = (contentWeight, styleWeight)
  
  # Defining the Config File
  config = {
    'model': model,
    'lossWeights': lossWeights,
    'generateImage': generateImage,
    'contentFeatures': contentFeatures,
    'styleFeatures': styleFeatures
    }
  
  globalStart = time.time()
  
  normMeans = np.array([103.939, 116.779, 123.68])
  minVals = -normMeans
  maxVals = 255 - normMeans  

  # Creating Logs to use for Plotting Later
  contentCostLog, styleCostLog, totalCostLog = [], [], []

  ETA = 0
  timeLog = []

  PATH = f'/content/drive/My Drive/Colab Notebooks/Neural Style Transfer/output/{output_dirName}'
  if not os.path.isdir(PATH):
    os.mkdir(PATH)
  os.chdir(PATH)

  for iter in range(iterations):
    startTime = time.time()

    # Computing the Grads and Loss
    grads, allLoss = computeGrads(config)

    # Extracting different kinds of Losses
    loss, contentLoss, styleLoss = allLoss
    
    # Saving the respective losses in respective lists for plotting
    contentCostLog.append(contentLoss)
    styleCostLog.append(styleLoss)
    totalCostLog.append(loss)

    # Applying gradients to Generate Image
    optimizer.apply_gradients([(grads, generateImage)])

    # Clipping the values of Generate Image from (-255, 255)
    clipped = tf.clip_by_value(generateImage, minVals, maxVals)
    generateImage.assign(clipped)

    endTime = time.time()

    timeTakenbyOneImage = endTime - startTime
    timeLog.append(timeTakenbyOneImage)

    ETA = statistics.mean(timeLog) * (iterations-iter)

    # Updating the Best Image and Loss
    if loss < bestLoss:
      bestLoss = loss
      bestImage = generateImage.numpy()

    # Saving the Generate Image
    if iter % SAVE_EVERY == 0:
      new = cv2.cvtColor(deprocessImage(generateImage.numpy()), cv2.COLOR_RGB2BGR)
      cv2.imwrite(f'generateImage_{iter+1}.jpg', new)

      print(f"Image {iter+1} Saved || Time Taken: {round(timeTakenbyOneImage, 2)} sec || ETA: {round(ETA/60, 2)} min" )
    

  # Saving the numpy Arrays to plot later
  np.save('contentLoss.npy', contentCostLog)
  np.save('styleLoss.npy', styleCostLog)
  np.save('totalCostLoss.npy', totalCostLog)

  print('Total Time taken in training :', round(endTime-globalStart, 2))

  return bestImage, bestLoss, output_dirName


## Visualizing our Content And Style Images

In [0]:
'''
Style Templates:
  > Wave : https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ8LOCDfTvcJ_V4fBdtL3R_oQn7D9P96PPzJFCksdWeKHHhyfUZ
  > Seated Nude : https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ0ip7KMW5XB_qhU3cwBDDd1fjlogHfgOxw9gnVq2CqZdLwHgY3
'''

# Enter the path of the respective Images
contentImagePath = '/content/drive/My Drive/Colab Notebooks/Neural Style Transfer/images/kingStarry.jpg'
styleImagePath   = '/content/drive/My Drive/Colab Notebooks/Neural Style Transfer/images/style5.jpg'

content, var = inputImageAndPreprocess(contentImagePath)
style, var   = inputImageAndPreprocess(styleImagePath)

plt.figure(figsize=(10, 10))

plt.subplot(1, 2, 1)
plt.imshow(content)
plt.title('Content Image')

plt.subplot(1, 2, 2)
plt.imshow(style)
plt.title('Style Image')

# **Training**

In [0]:
ITERATIONS = 2500

bestImage, bestLoss, output_dirName = runStyleTransfer(contentImagePath,
                                                      styleImagePath,
                                                      iterations=ITERATIONS,
                                                      SAVE_EVERY = 5,
                                                      contentWeight = 1,
                                                      styleWeight=0.8,
                                                      output_dirName = 'kingStarry')

# Output Cleared for Fairness

#### Visualizing the Best Image

In [0]:
plt.figure(figsize=(10,10))
plt.imshow(deprocessImage(bestImage))
plt.title('Generated Image')

#### Plotting the Cost Functions

In [30]:
PATH = f'/content/drive/My Drive/Colab Notebooks/Neural Style Transfer/output/{output_dirName}'
os.chdir(PATH)
contentLog = np.load('contentLoss.npy')
print('Content Log Loaded..!')
styleLog = np.load('styleLoss.npy')
print('Style Log Loaded..!')
totalLog = np.load('totalCostLoss.npy')
print('Total Loss Log Loaded..!')
iterations = list(range(1, len(styleLog)+1))

Content Log Loaded..!
Style Log Loaded..!
Total Loss Log Loaded..!


In [0]:
plt.figure(figsize=(23, 8))
plt.plot(totalLog,   linewidth=3, label='total loss')
plt.plot(styleLog,   linewidth=1, label='style loss')
plt.plot(contentLog, linewidth=2, label='content loss')
# plt.plot(learning_curve_tv, linewidth=2, label='total variation loss')
plt.title("Learning curve")
plt.ylabel("error")
plt.xlabel("epoch")
plt.yscale("log")
plt.legend()
plt.grid()
plt.show()